In [18]:
# 定义几个名字
experiment_name = "DiscPC的baseline实验-std测试-SGD"
user_name = "mengfan"
save_dir = "discpc_baseline_SGD-std.pkl"

In [ ]:
from torch.nn.modules import activation
from radas import run_experiment, get_ray_results_dir  # pyright: ignore[reportMissingImports]
import ray.tune as tune
from DiscPC_SGD import trainable
import torch
import pandas as pd
import numpy as np


results = await run_experiment(
    user_name=user_name,
    experiment_name=experiment_name,
    param_space=dict(
        seed= tune.grid_search(list[int](range(100))),
        device=None,
        download=False,
        batch_size=256,
        epochs=25,
        activation="leaky_relu",
        last_send_activation="identity",

        # 关键：推断别太弱（DiscPC/PC里这两项对 SGD 尤其关键）
        steps_train=8,
        steps_train_max=8,
        steps_train_schedule="constant",
        lr_x=0.03,
        momentum_x=0.9,

        steps_test=100,

        # mini-batch SGD on theta
        lr_theta=1e-3,
        momentum_theta=0.9,
        weight_decay=1e-2,
        grad_clip_norm=0.0,

        # 关键：先 constant，避免“调参时你以为lr是常数，其实在衰减”
        lr_theta_schedule="constant",     # constant|cosine|cosine_warmup
        lr_theta_min=0.0,
        lr_theta_warmup_steps=0,

        label_mode="onehot_01",
        log_every=5,
        num_workers=0,
        pin_memory=False,
        weight_init="xavier",
        user_name="mengfan",
    ),
    # IMPORTANT: when using cluster, your trainable cannot be defined in this script, but has to be defined in a separate file, and imported in this script
    trainable=trainable,
    # When use cluster, local_storage_path needs to be specified in this way
    local_storage_path=get_ray_results_dir(),
    # Specify the cluster to use
    run_with="cluster:atol-gpu-5090",
    # Specify gram per trial
    cpu_per_trial=1,
    # If the gram_per_trial you set is larger than GRAM of a single GPU, you will be allocated with more than one GPUs
    # to fit your need. But in this case, you will need to handle how do you want to distribute your workload to these multiple GPUs
    # To check GRAM of a single GPU, use the following code:
    # from radas.clusters import gram_per_gpu
    #
    runtime_env={
        "pip": ["emoji","scikit-learn"],
        # for more runtime_env specifications, see https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#api-reference
    },
    #
    instant_gpu_allocation=1,
)

[INFO]: Checking if the experiment exists...
[Choices]: Choose an option by number:
[1]: run -- Run a new experiment (if there is a running experiment, it will be stopped; if there is a existing experiment in the storage, it will be removed)
 2 : restore -- Restore an existing experiment from the storage (if there is a running experiment, it will be stopped)
 3 : attach -- Attach to a running experiment
 4 : stop_attach -- Stop and attach to a running experiment
>>> [1]: Run a new experiment (if there is a running experiment, it will be stopped; if there is a existing experiment in the storage, it will be removed)
[INFO]: 🚀 正在测试代理连接...
[INFO]: ✅ 代理连接成功


2025-12-26 21:54:07,574	INFO dashboard_sdk.py:355 -- Uploading package gcs://_ray_pkg_12689530c683fafc.zip.
2025-12-26 21:54:07,575	INFO packaging.py:588 -- Creating a file package for local module '/mnt/data/home/mengfan/projects/BiPCN/BiPCN/DiscPC_Baseline'.
2025-12-26 21:54:07,718	INFO dashboard_sdk.py:402 -- Package gcs://_ray_pkg_4588c91f8f921f3a.zip already exists, skipping upload.


[INFO]: Submitting job...
[INFO]: Job submitted.
[INFO]: Tailing experiment logs below 👇👇👇
2025-12-26 08:54:07,735	INFO job_manager.py:531 -- Runtime env is setting up.

  0%|          | 0/1204 [00:00<?, ?file/s]
rm_local_folder: mengfan/DiscPC的baseline实验-std测试-SGD/trainable_ccb07_00001_1_seed=1_2025-12-26_08-50-12/params.pkl:   0%|          | 0/1204 [00:00<?, ?file/s]
rm_local_folder: mengfan/DiscPC的baseline实验-std测试-SGD/trainable_ccb07_00001_1_seed=1_2025-12-26_08-50-12/events.out.tfevents.1766757024.atol-gpu-5090-head:   0%|          | 0/1204 [00:00<?, ?file/s]
rm_local_folder: mengfan/DiscPC的baseline实验-std测试-SGD/trainable_ccb07_00001_1_seed=1_2025-12-26_08-50-12/result.json:   0%|          | 0/1204 [00:00<?, ?file/s]                                      
rm_local_folder: mengfan/DiscPC的baseline实验-std测试-SGD/trainable_ccb07_00001_1_seed=1_2025-12-26_08-50-12/params.json:   0%|          | 0/1204 [00:00<?, ?file/s]
rm_local_folder: mengfan/DiscPC的baseline实验-std测试-SGD/trainable_ccb07_000

In [20]:
results["df"]
df = results["df"]

In [21]:
# 2. 一键保存为pkl文件（index=False 不存行索引，更简洁）
df.to_pickle(save_dir, protocol=4)  # protocol=4 兼容多数Python版本，读写更快

In [22]:
results["df"]


,device,torch.cuda.is_available,config/seed,config/batch_size,config/epochs,config/layers,config/activation,config/last_send_activation,config/steps_train,config/steps_train_max,...,final/E_test_after_infer,training_iteration,config/experiment_name,config/device,config/download,config/log_every,config/num_workers,config/pin_memory,config/weight_init,config/user_name
0,cuda,True,89,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.057456,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
1,cuda,True,93,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.056721,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
2,cuda,True,13,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.055811,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
3,cuda,True,27,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.056851,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
4,cuda,True,80,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.056569,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,cuda,True,88,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.056477,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
96,cuda,True,20,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.056784,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
97,cuda,True,44,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.057021,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
98,cuda,True,76,256,25,"[784, 256, 256, 10]",leaky_relu,identity,8,8,...,0.055679,1,DiscPC的baseline实验-std测试-SGD,None,False,5,0,False,xavier,mengfan
